In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 15

# Wikilabels
df = pd.read_csv('../data/wiki/wiki_name_race.csv')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df

# Additional features
sdf['name_first'] = sdf.name_first.str.title()

sdf.groupby('race').agg({'name_first': 'count'})

,name_first
race,
"Asian,GreaterEastAsian,EastAsian",5497
"Asian,GreaterEastAsian,Japanese",7333
"Asian,IndianSubContinent",7861
"GreaterAfrican,Africans",3672
"GreaterAfrican,Muslim",6242
"GreaterEuropean,British",41445
"GreaterEuropean,EastEuropean",8329
"GreaterEuropean,Jewish",10239
"GreaterEuropean,WestEuropean,French",12293


## Preprocessing the input data

In [2]:
# only last name will be use to train the model
sdf['name_last_name_first'] = sdf['name_last'] 

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1435


/opt/venv/lib/python2.7/site-packages/ipykernel_launcher.py:28: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


Max feature len = 71, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [3]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

/opt/venv/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


107097 train sequences
26775 test sequences
Pad sequences (samples x time)
X_train shape: (107097, 20)
X_test shape: (26775, 20)
13 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (107097, 13)
y_test shape: (26775, 13)


In [4]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            45920     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 13)                1677      
Total params: 130,029
Trainable params: 130,029
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 96387 samples, validate on 10710 samples
Epoch 1/15
 - 157s - loss: 1.6567 - acc: 0.4978 - val_loss: 1.4391 - val_acc: 0.5661
Epoch 2/15
 - 182s - loss: 1.4154 - acc: 0.5791 - val_loss: 1.3618 - val_acc: 0.5984
Epoch 3/15
 - 200s - loss: 1.3502 - acc: 0.6010 - val_loss: 1.3151 - val_acc: 0.6143
Epoch 4/15
 - 199s - loss: 1.3041 - acc: 0.6166 - val_loss: 1.2869 - val_acc: 0.6274
Epoch 5/15
 - 205s - loss: 1.2706 - acc: 0.6248 - val_loss: 1.2664 - val_acc: 0.6315
Epoch 6/15
 - 229s - loss: 1.2436 - acc: 0.6334 - val_loss: 1.2538 - val_acc: 0.6349
Epoch 7/15
 - 240s - loss: 1.2199 - acc: 0.6413 - val_loss: 1.2354 - val_acc: 0.6415
Epoch 8/15
 - 243s - loss: 1.2002 - acc: 0.6478 - val_loss: 1.2247 - val_acc: 0.6434
Epoch 9/15
 - 244s - loss: 1.1859 - acc: 0.6520 - val_loss: 1.2107 - val_acc: 0.6494
Epoch 10/15
 - 252s - loss: 1.1706 - acc: 0.6570 - val_loss: 1.2038 - val_acc: 0.6518
Epoch 11/15
 - 270s - loss: 1.1570 - acc: 0.6608 - val_loss: 1.2030 - val_acc: 0.6496
Epoc

## Confusion Matrix

In [6]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

                                       precision    recall  f1-score   support

     Asian,GreaterEastAsian,EastAsian       0.81      0.76      0.78      1099
      Asian,GreaterEastAsian,Japanese       0.82      0.87      0.84      1467
             Asian,IndianSubContinent       0.67      0.66      0.67      1572
              GreaterAfrican,Africans       0.52      0.36      0.42       734
                GreaterAfrican,Muslim       0.54      0.50      0.52      1248
              GreaterEuropean,British       0.70      0.87      0.78      8289
         GreaterEuropean,EastEuropean       0.74      0.64      0.69      1666
               GreaterEuropean,Jewish       0.44      0.34      0.39      2048
  GreaterEuropean,WestEuropean,French       0.57      0.46      0.51      2459
GreaterEuropean,WestEuropean,Germanic       0.40      0.26      0.32       774
GreaterEuropean,WestEuropean,Hispanic       0.62      0.50      0.56      2082
 GreaterEuropean,WestEuropean,Italian       0.63   

## Save model

In [11]:
model.save('./models/wiki/lstm/wiki_ln_lstm.h5')

In [12]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/wiki/lstm/wiki_ln_vocab.csv', index=False, encoding='utf-8')